In [21]:
import gym
import numpy as np
import time,math
import matplotlib.pyplot as plt
import typing
import cv2 as cv

In [4]:
import gym
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML


In [5]:
path = "Cart-pole/weights"

**Here we are Training our Taxi**

***After certain iterations our Taxi will Learn to drop and pick-up passangers perfectly***

In [7]:
# Making our env 
env_taxi = gym.make('Taxi-v3',new_step_api='True',render_mode='rgb_array').env
alpha = 0.5
gamma = 0.5

In [8]:
# We will use this fn to update our Q table
def update_Q_Table(Q,s,a,s2,r):
  Q[s,a] = alpha*( r + gamma*np.max(Q[s2])  - Q[s,a] )  

In [16]:
# Creating our Empty Q table here
n_states = env_taxi.observation_space.n
n_actions = env_taxi.action_space.n
Q_taxi = np.zeros((n_states,n_actions))
print(f' action space : {n_actions} \n state space : {n_states}')

 action space : 6 
 state space : 500


In [71]:
# this is the policy ; based on which we will take action
def policy(s,Q,i):
  rand = np.random.rand()
  k = 0.8 if i<100 else max(1/i,0.01)
  if rand <= k:
    action = env_taxi.action_space.sample()
  else:
    action = np.argmax(Q[s])
  return action

def train_taxi(Q,epochs=5000):
  reward = 0
  for i in range(1,epochs):
    s = env_taxi.reset()

    for j in range(1000):

      action = policy(s=s,Q=Q,i=i)
      s2 , reward , done , info , _ = env_taxi.step(action)
      Q[s,action] = alpha*( reward + gamma*np.max(Q[s2])  - Q[s,action] )  
      #update(Q,s,action,s2,reward)
      s = s2

      if done == True:
        break
      
    if i%5 == 0:
      print(f'epoch {i} done ')

  env_taxi.close()
  return Q

In [72]:
Q_taxi = train_taxi(Q_taxi,50)

epoch 5 done 
epoch 10 done 
epoch 15 done 
epoch 20 done 
epoch 25 done 
epoch 30 done 
epoch 35 done 
epoch 40 done 
epoch 45 done 


In [76]:
print(Q_taxi)

[[ 0.          0.          0.          0.          0.          0.        ]
 [-0.4        -0.39999989 -0.39999998 -0.39999993 -0.39999927 -3.40000006]
 [-0.39996974 -0.39983013 -0.39996631 -0.39983296 -0.39908206 -3.39988197]
 ...
 [-0.3990873  -0.39454715 -0.39894157 -0.39977747 -3.39905845 -3.3990862 ]
 [-0.39999923 -0.39999525 -0.39999869 -0.3999935  -3.39999828 -3.4000003 ]
 [-0.20334395 -0.36511661 -0.20360439  0.77777708 -3.20299149 -3.21139652]]


In [95]:
# testing our Q Learning
def test_Q_taxi(Q,path='./Cart-pole , taxi , mountain car video/taxi-video-5.avi'):
  state = env_taxi.reset()
  done = False
  i=0

  # Capturing video of taxi
  video_converter =  cv.VideoWriter_fourcc(*'mp4v')
  video_taxi = cv.VideoWriter(path,video_converter,4,(550,350))

  while done!=True and i<90:
    
    action = np.argmax(Q[state])
    state,reward,done,info,_ = env_taxi.step(action)
    output_img = np.array(env_taxi.render())[-1]
    #print(output_img[-1].shape)

    video_taxi.write(output_img)
    i+=1
  print(f'video has been saved in the path : {path} and total iters : {i} \n final reward : {reward}')

In [96]:
test_Q_taxi(Q_taxi)

video has been saved in the path : ./Cart-pole , taxi , mountain car video/taxi-video-5.avi and total iters : 10 
 final reward : 20


In [97]:
# Environment - 2: Mountain car

env2 = gym.make('MountainCar-v0',new_step_api='True',render_mode='rgb_array').env
alpha2 = 0.2
gamma2 = 0.9

Q_dimensions = np.array([env2.observation_space.high - env2.observation_space.low])*np.array([10,100])
dims = np.int_(np.round(Q_dimensions,0)+1)
dims = list(np.append(dims[0],3))
print(dims)
Q2 = np.zeros(dims)
Q2.shape

[19, 15, 3]


(19, 15, 3)

In [99]:
# Fn to already learned load Q table for environments
def load_Q_table(delimiter=' ',path='./Cart-pole weights/Q-value-mountain-car.csv'):
    Q = np.loadtxt(path,delimiter=delimiter)
    return Q
Q2 = load_Q_table()

In [103]:
print(Q2.shape)
Q2 = Q2.reshape(19,15,3)
print(Q2)

(19, 15, 3)
[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 1.15381134e-02  1.15217508e-02 -7.39302329e-03]
  [ 1.15621461e-02  1.15329566e-02  1.15329594e-02]
  [ 1.15328652e-02  1.15292827e-02  1.15342201e-02]
  [ 1.15308880e-02  1.15349721e-02  1.14950414e-02]
  [ 1.15620720e-02  1.15338980e-02  1.15316023e-02]
  [ 1.15333457e-02  1.15301753e-02  1.16898761e-02]
  [ 1.16906196e-02  1.15167084e-02  1.15129431e-02]
  [ 1.25825265e-02  1.15122875e-02  1.15130492e-02]
  [ 1.22465662e-02  1.15161364e-02  1.24417457e-02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 1.10610546e-02  1.15628559e-02  1.13944468e-02]
  [ 1.13871048e-02  1.13899225e-02  1.22318348e-02]
  [ 1.20994061e-02  1.21995316e-02  1.22552617e-02

In [104]:
# Our policy
def policy_m(s,Q,i):
  rand = np.random.rand()
  k = 0.33-i/100
  if rand <= k:
    action = env2.action_space.sample()
  else:
    action = np.argmax(Q[s[0],s[1]])
  return action

In [105]:
# map_states fn to discretesize the continuous space
def map_state(s):
  s[0] = (1.2+s[0])*10
  s[1] = (0.07+s[1])*100
  '''s += [1.2,0.07]
  s = s*np.array([10,100]) 
  s = np.int_(s)'''
  return s.astype(int)

In [107]:
# checking , the extreme values
print( map_state( np.array([-1.2,-0.07]) ) )

[0 0]


In [108]:
# My custom defined reward fn to make Ai learn more quicker
def R(rew):
  y = float(+16 - rew[0])
  y = 100 if y <= 0 else (1+y)**-1
  return y

In [109]:
k = np.array([1,1])
k = np.int_(k)
R(k)

0.0625

In [ ]:
stat_m1 = env2.reset()

for i in range(1,30):
  done2 = 0
  stat_m1 = env2.reset()
  stat_m1 = map_state(stat_m1)
  j = 0
  print(i,end=' ')
  while done2!=True:

    action2 = policy_m(s=stat_m1,Q=Q2,i=i)
    stat_m2 , reward2 , done2 , info2 = env2.step(action2)
    stat_m2 = map_state(stat_m2) # mapping state to the Q2 table
    reward2 = R(stat_m2)

    Q2[stat_m1[0],stat_m1[1],action2] = alpha2*( reward2 + gamma2*np.max(Q2[stat_m2[0],stat_m2[0]])  - Q2[stat_m1[0],stat_m1[1],action2] )  
    #update(Q,s,action,s2,reward)

    stat_m1 = stat_m2 # changing the states

    '''if reward2 > 0:
      print(f'{reward2} in {action2} and {stat_m1[0]} ; {stat_m1[1]}')'''

    if done2 == True and stat_m2[0] >= 17:
      print(stat_m2[0],'>',reward2,'>',stat_m2[1],end=';')
      Q2[stat_m2[0], stat_m2[1], action2] = alpha2*reward2 - alpha2*Q2[stat_m2[0], stat_m2[1], action2]
      break
    j +=1

env2.close()

1 17 > 100 > 8;2 17 > 100 > 8;3 17 > 100 > 7;4 17 > 100 > 7;5 17 > 100 > 7;6 17 > 100 > 7;7 17 > 100 > 8;8 17 > 100 > 8;9 17 > 100 > 7;10 17 > 100 > 8;11 17 > 100 > 8;12 17 > 100 > 7;13 17 > 100 > 7;14 17 > 100 > 8;15 17 > 100 > 8;16 17 > 100 > 9;17 17 > 100 > 7;18 17 > 100 > 9;19 17 > 100 > 7;20 17 > 100 > 7;21 17 > 100 > 7;22 17 > 100 > 8;23 17 > 100 > 7;24 17 > 100 > 7;25 17 > 100 > 7;26 17 > 100 > 9;27 17 > 100 > 9;28 17 > 100 > 9;29 17 > 100 > 9;

In [112]:
print(Q2,Q2.shape,sep="\n")

[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 1.15381134e-02  1.15217508e-02 -7.39302329e-03]
  [ 1.15621461e-02  1.15329566e-02  1.15329594e-02]
  [ 1.15328652e-02  1.15292827e-02  1.15342201e-02]
  [ 1.15308880e-02  1.15349721e-02  1.14950414e-02]
  [ 1.15620720e-02  1.15338980e-02  1.15316023e-02]
  [ 1.15333457e-02  1.15301753e-02  1.16898761e-02]
  [ 1.16906196e-02  1.15167084e-02  1.15129431e-02]
  [ 1.25825265e-02  1.15122875e-02  1.15130492e-02]
  [ 1.22465662e-02  1.15161364e-02  1.24417457e-02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 1.10610546e-02  1.15628559e-02  1.13944468e-02]
  [ 1.13871048e-02  1.13899225e-02  1.22318348e-02]
  [ 1.20994061e-02  1.21995316e-02  1.22552617e-02]
  [ 1.1390

In [149]:
# testing our Q Learning for mountain car
def test_Q_mountain_car(Q,path='./Cart-pole , taxi , mountain car video/mountain-car-video-2.avi',epochs=1):

  # Capturing video of car
  video_converter =  cv.VideoWriter_fourcc(*'mp4v')
  video2 = cv.VideoWriter(path,video_converter,60,(600,400))

  for _ in range(epochs):

    done  = False
    i = 0
    state = env2.reset()

    while done!=True and i<500:
      
      state = map_state(state)
      action = np.argmax(Q[state[0],state[1]])
      state,reward,done,info,_ = env2.step(action)

      output_img = np.array(env2.render())[-1]
      #print(output_img.shape)
      
      video2.write(output_img)
      i+=1

  video2.release()
  print(f'video has been saved in the path : {path} and total iters : {i} \n final reward : {reward} ')

In [150]:
test_Q_mountain_car(Q2,epochs=5)

video has been saved in the path : ./Cart-pole , taxi , mountain car video/mountain-car-video-2.avi and total iters : 500 
 final reward : -1.0 


In [151]:
# DQN in cartpole

import os
import glob as gb
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import time
from IPython.display import clear_output

In [155]:
# Creating our DQNetworks

class DeepQNet(nn.Module):
  def __init__(self,input_dims=6,output_dims=2):

    super(DeepQNet,self).__init__()
    
    self.input_dims = input_dims
    self.output_dim = output_dims

    #  Model Here
    self.dense_layer = nn.Sequential(
        nn.Linear(4, 32),
        nn.LeakyReLU(),
        nn.Linear(32,64),
        nn.LeakyReLU(),
        nn.Linear(64,28),
        nn.LeakyReLU(),
        nn.Linear(28,self.output_dim)
    )

  def forward(self,x):
      y = self.dense_layer(x)
      return y

In [156]:
model_cart_pole = DeepQNet().to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# About Model
def about_model(model):
    print(model)
    print(f' total params in the model : {sum(p.numel() for p in model.parameters())}' )
    
about_model(model_cart_pole)

DeepQNet(
  (dense_layer): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=64, out_features=28, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=28, out_features=2, bias=True)
  )
)
 total params in the model : 4150


In [ ]:
class Replay_Buffer():
  def __init__(self,max_size,input_shape,action_space):
    self.memory_size = max_size
    self.mem_flow = 0
    self.mem_count = 0

    # defining the data structure to store the replay
    self.state_mem = np.zeros((self.memory_size,*input_shape),dtype=np.float32)
    self.state2_mem = np.zeros((self.memory_size,*input_shape),dtype=np.float32)
    self.rewards = np.zeros(self.memory_size,dtype=np.float32)
    self.actions = np.zeros((self.memory_size,*action_space),dtype=np.int64)

    # fn to store replay
  def store_replay(self,state,action,reward,state2):
      self.mem_count = 0 if self.mem_count>=self.memory_size else self.mem_count
      ind = self.mem_count  #% self.memory_size # to overcome dataoverflow

      if state.shape != (4,) or state2.shape != (4,) or action.shape != (2,) or type(reward) != float:
        raise Exception(f"Sorry, wrong dims {state.shape} and {state2.shape} {type(action)}  and {type(reward)} and {action.shape}")
      state = state.astype(float)
      state2 = state.astype(float)

      self.state_mem[ind] = state
      self.state2_mem[ind] = state2
      self.rewards[ind] = reward
      self.actions[ind] =  action
      self.mem_count += 1

  def get_sample(self,batch_size=100):
    max_mem = min(self.mem_count,self.memory_size) # to ensure we have enough data

    if batch_size >= max_mem:
      raise Exception(f'{batch_size} is given but we have only {max_mem} this much samples , reduce batch size')

    else :
      batch = np.random.choice(max_mem,batch_size,replace=False)
      state1 =  self.state_mem[[batch]]
      state2 =  self.state2_mem[[batch]]
      reward =  self.rewards[[batch]]
      action1 = self.actions[[batch]]
    
      return [state1,action1,reward,state2]

  def get_info(self):
    print(f"The dims of state1_mem is {self.state_mem.shape}")
    print(f"The dims of state2_mem is {self.state2_mem.shape}")
    print(f"The dims of reward is {self.rewards.shape}")
    print(f"The dims of actions is {self.actions.shape}")
    print(f"they are populated upto {self.mem_count}")
      

In [ ]:
def tarin_model(data,model1,model2,gamma=0.90,lr=0.01,loss_fn = nn.MSELoss(),optimizer=torch.optim.Adam):

      train_loss = []
      act = np.array([[0,1],[1,0]])
      optimizer = optimizer(model1.parameters(),lr=lr)


      '''def action_to_vec(a,n_actions):
        for i  in range(len(a)):
          k = np.zeros(len(a),n_actions)
          k[i,a] = 1
        return k'''

      # extracting data
      state1 = data[0]
      actions = data[1]
      rewards = data[2]
      state2 = data[3]

      #actions = action_to_vec(data[1],n_acts) 

      # making state and action as pairs
      data2 = np.zeros( (len(data[1]),6) )
      data3 = np.zeros( (len(data[1]),2,6) , dtype=np.float32)
      for i in range(len(data[1])):
        #print(len(data))
        data2[i] = np.concatenate((data[0][i],actions[i]))
        data3[i] = np.array([  np.concatenate((data[3][i],act[0])) , np.concatenate((data[3][i],act[1]))   ] , dtype = np.float32)
      #print(data2,' \n \n \n',data3)


      # changing data to cuda device
      device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
      model2.to(device)
      model1.to(device)
      torch_data_Q = torch.Tensor(data2).to(device)
      torch_data_T = torch.Tensor(data3).to(device)


      # getting the y_true values
      y_true = torch.empty(len(data[1]),1)

      for i,v in enumerate(torch_data_T):
        y_t = np.array(max(list(  model2(v[i]).item()  for i in range(2))),dtype=np.float64)
        y_true[i] = torch.from_numpy(np.array([(rewards[i] + gamma*y_t)])).to(device)
      y_true = y_true.to(device)

      loss_fn = loss_fn.to(device)

      # Training the model
      optimizer.zero_grad()
      y_pred = model1(torch_data_Q)
      #print(f'\n \n \n y_pred {y_pred} \n \n, ytrue\n{y_true} ')
      #raise Exception('for testing')
      loss = loss_fn(y_pred,y_true)
      loss.backward()
      optimizer.step()
      train_loss.append(loss.item())


In [ ]:
def collect_replay(env,replay,Q_model,eps,d_rate = 5e-4,j=2000):

  state = env.reset()
  action_space = np.array( [[0,1],[1,0]] )
  device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  Q_model.to(device)

  while replay.mem_count <(j):
      rand = np.random.rand()
      eps = eps - d_rate if eps > 0.01 else 0.01

      if rand <= eps:
        #print(i,end='+')
        action = np.array(env.action_space.sample())
        state2,reward,done,info = env.step(action)
        state2 = np.array(state2)
        if done == True:
          state = np.array(env.reset())
          #print(0,i)
          continue
        replay.store_replay(state,action_space[action],reward,state2)
        state = state2

      else:
        #print(j,end='*')
        state_pair = list(torch.Tensor(np.concatenate((state,action_space[i]))).to(device) for i in range(2) )
        action = np.argmax(np.array(list( Q_model(state_pair[i]).item() for i in range(2))) )
        state2,reward,done,info = env.step(action)
        state2 = np.array(state2)
        #print(i,end='p')
        if done == True:
          state = np.array(env.reset())
          continue
        replay.store_replay(state,action_space[action],reward,state2)
        state = state2
  return eps

In [ ]:
def transfer_weight(model1,model2,i):
  if i%2 == 0:
    model2.load_state_dict(model1.state_dict())

In [ ]:
class agent():
  def __init__(self,modelname,max_mem=2000,freq=2,decay=1e-5):

    self.modelname = modelname
    self.freq_weight_transfer =  freq
    self.max_mem = max_mem
    self.env = gym.make('CartPole-v0').env
    self.state = np.array(self.env.reset())
    self.eps = 0.8
    self.decay = decay

    self.Q_model = DeepQNet()
    self.Trainer = DeepQNet()
    self.replay = Replay_Buffer(self.max_mem,(4,),(2,))
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  def get_experience(self,j=2000,info_need=0): # storing replay
        j = self.max_mem if self.max_mem < j else j
        self.eps = collect_replay(self.env,self.replay,self.Q_model,self.eps,d_rate =self.decay,j=j)
        if info_need:
          self.replay.get_info()


  def train_from_experience(self,epochs=15,batch_size=100):  # train_model
    if epochs < self.max_mem/batch_size:
        for i in range(epochs):
            dataset = self.replay.get_sample(batch_size)
            tarin_model(model1=self.Q_model , model2=self.Trainer , data=dataset)
    else:
      print(f" max epoch possible is {self.max_mem/batch_size} but you have given {epochs}")


  def transfer_weight(self,i,show='no'):
      if i%self.freq_weight_transfer == 0:
        if show == 'yes':
          print(self.Q_model.state_dict()['dense_layer.0.weight'],'model2 \n \n \n',self.Trainer.state_dict()['dense_layer.0.weight'])
        self.Trainer.load_state_dict(self.Q_model.state_dict())
        
  def save_load_model(self,key='save'): # To load and save model
     path  = f'/content/gdrive/MyDrive/RL_Model_weights/{self.modelname}.pt'

     if key == 'save':
        print(f"...Saving Model...")
        torch.save(self.Q_model.state_dict(), path )
     elif key == 'load':
       print(f"...Loading Model...")
       self.Q_model.load_state_dict(torch.load(path,map_location=self.device))
     else:
       print(f"the key {key} is invalid")
    

  # transfer_weight
  # transfer_weight(Q_model,Trainer,i)

In [ ]:
# checking the model
A = agent('new-1')
#B = agent('1')

In [ ]:
#Checking Functionality
A.transfer_weight(2)
A.get_experience()
A.train_from_experience(batch_size=1)
#A.transfer_weight(8,'yes')
A.save_load_model()
'''
B.save_load_model(key='load')
B.transfer_weight(2)
B.transfer_weight(8,'yes')'''

...Saving Model...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a differe

"\nB.save_load_model(key='load')\nB.transfer_weight(2)\nB.transfer_weight(8,'yes')"

In [ ]:
# Real Training begings Here

def train_Ai(epochs):
  AI_1 = agent(modelname='DQN5-cartpole',max_mem=10000,freq=100)
  AI_1.transfer_weight(100)
  AI_1.save_load_model('load')
  for i in range(epochs+1):
    AI_1.get_experience(j=1000)
    AI_1.train_from_experience(epochs=990,batch_size=10)
    AI_1.transfer_weight(i)
    if i%10 == 0:
      print(i,end=' * ')
      AI_1.save_load_model()


In [ ]:
train_Ai(300)

...Loading Model...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a differe

0 * ...Saving Model...
10 * ...Saving Model...
20 * ...Saving Model...
30 * ...Saving Model...
40 * ...Saving Model...
50 * ...Saving Model...
60 * ...Saving Model...
70 * ...Saving Model...
80 * ...Saving Model...
90 * ...Saving Model...
100 * ...Saving Model...
110 * ...Saving Model...
120 * ...Saving Model...
130 * ...Saving Model...
140 * ...Saving Model...
150 * ...Saving Model...
160 * ...Saving Model...
170 * ...Saving Model...
180 * ...Saving Model...
190 * ...Saving Model...
200 * ...Saving Model...
210 * ...Saving Model...
220 * ...Saving Model...
230 * ...Saving Model...
240 * ...Saving Model...
250 * ...Saving Model...
260 * ...Saving Model...
270 * ...Saving Model...
280 * ...Saving Model...
290 * ...Saving Model...
300 * ...Saving Model...


In [ ]:
k = torch.Tensor([[1,2,3,4,5,6,7]])
l = torch.empty(10)
for i in range(10):
  l[i] = torch.tensor(i) 
print(l.shape)

torch.Size([10])


In [ ]:
env = gym.make('CartPole-v0').env
env.reset()
action = np.argmax(np.array([1,2,3]))
state2,reward,done,info = env.step(1)
print(type(action))

<class 'numpy.int64'>


In [ ]:
k22  = np.array([1,1,1])
np.argmax(k22)

0

In [ ]:
def best_action(AI,actions,state,device):
 a1 = torch.Tensor(np.concatenate((state,actions[1]))).to(device)
 a2 = torch.Tensor(np.concatenate((state,actions[0]))).to(device)
 acts = [a1,a2]
 #print(a1.shape)
 best_action = np.argmax(np.array([ AI(acts[i]).item()  for i in range(2)]))
 return best_action


def enhance(Ai,trained_ai_name,device):
  Ai.load_state_dict(torch.load(path,map_location=torch.device(device)))
  return Ai

In [ ]:
def predict(model):
  ai = DeepQNet()
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  ai = enhance(ai,model,device)
  #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  acts = np.array( [[0,1],[1,0]] )
  #print(ai.state_dict()['dense_layer.0.weight'])
  ai.to(device)
  env = gym.make('CartPole-v0').env
  state = np.array(env.reset())
  done = False
  i=0
  while done != True:
      i+=1
      action =  best_action(AI=ai,actions=acts,state=state,device=device)
      state2,reward,done,info = env.step(action)
      state = np.array(state2)
      if done == True:
        print(state,i)

In [ ]:
predict('DQN2-cartpole1')

[ 0.17862858  1.72158552 -0.24517753 -2.84320287] 9


In [ ]:
env22 = gym.make('CartPole-v0').env
env22.reset()

Object `env22.env22` not found.
